<a href="https://colab.research.google.com/github/poziryna84/Dr_Cervantes_AI_Saturday_Madrid/blob/master/Sintomas_medicos_modelado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

### Cargamos los datos.

In [0]:
med_df = pd.read_excel('/content/drive/My Drive/Datos_medicos/datos_medicos.xlsx')

In [18]:
med_df.columns

Index(['Unnamed: 0', 'IdIngreso', 'NºHISTORIAL', 'FECHA_ALTA', 'SERVICIO',
       'MOTIVO_CONSULTA', 'ENF_ACTUAL', 'EXPLORACION1', 'ANALITICA1',
       'EXPL_DIAGNOSTICAS1', 'INTERVENCIONES1', 'EvolucionComentarios',
       'MotivoAlta', 'DiagnosticoPrincipalAlta', 'ServicioAlta',
       'DestinoAlta1', 'RecomendacionesTerapeuticas1', 'CODIGODIAG1', 'DIAG1',
       'aux_codigo', 'GENERAL_DIAG', 'SPECIFIC_DIAG'],
      dtype='object')

In [0]:
med_df['GENERAL_DIAG'] = med_df['GENERAL_DIAG'].apply(lambda x: ' '.join(x.split('. ')[1:]))

In [20]:
med_df['GENERAL_DIAG'].value_counts(normalize = True)

ENFERMEDADES DEL APARATO RESPIRATORIO (460-519)                                                  0.158381
ENFERMEDADES DEL SISTEMA OSTEO-MIOARTICULAR Y TEJIDO CONJUNTIVO (710-739)                        0.154886
ENFERMEDADES DEL SISTEMA CIRCULATORIO (390-459)                                                  0.132670
ENFERMEDADES DEL APARATO DIGESTIVO (520-579)                                                     0.105173
LESIONES Y ENVENENAMIENTOS (800-999)                                                             0.093910
NEOPLASIAS (140-239)                                                                             0.080006
ENFERMEDADES DEL APARATO GENITOURINARIO (580-629)                                                0.078841
SÍNTOMAS, SIGNOS Y ESTADOS MAL DEFINIDOS (780-799)                                               0.069675
ENFERMEDADES DEL SISTEMA NERVIOSO Y DE LOS ÓRGANOS DE LOS SENTIDOS (320-389)                     0.052276
ENFERMEDADES DE LA PIEL Y DEL TEJIDO SUBCUTÁNE

### Quitamos las etiquetas poco frequentes.

In [0]:
med_df = med_df[(med_df['GENERAL_DIAG'] != 'COMPLICACIONES DEL EMBARAZO, PARTO Y PUERPERIO (630-679)') &
(med_df['GENERAL_DIAG'] != 'ANOMALIAS CONGÉNITAS (740-759)') &
(med_df['GENERAL_DIAG'] != 'TRASTORNOS MENTALES, DEL COMPORTAMIENTO Y EL DESARROLLO NEUROLÓGICO (290-319)') &
(med_df['GENERAL_DIAG'] != 'ENFERMEDADES DE LA SANGRE Y DE LOS ORGANOS HEMATOPOYÉTICOS (280-289)') &
(med_df['GENERAL_DIAG'] != 'ENFERMEDADES ENDOCRINAS, DE LA NUTRICION Y METABOLICAS Y TRASTORNOS DE LA INMUNIDAD (240-279)') &
(med_df['GENERAL_DIAG'] != 'ENFERMEDADES INFECCIOSAS Y PARASITARIAS (001-139)') &
(med_df['GENERAL_DIAG'] != 'ENFERMEDADES DE LA PIEL Y DEL TEJIDO SUBCUTÁNEO (680-709)')]


In [143]:
med_df['GENERAL_DIAG'].value_counts(normalize = True)

ENFERMEDADES DEL APARATO RESPIRATORIO (460-519)                                 0.171071
ENFERMEDADES DEL SISTEMA OSTEO-MIOARTICULAR Y TEJIDO CONJUNTIVO (710-739)       0.167296
ENFERMEDADES DEL SISTEMA CIRCULATORIO (390-459)                                 0.143301
ENFERMEDADES DEL APARATO DIGESTIVO (520-579)                                    0.113600
LESIONES Y ENVENENAMIENTOS (800-999)                                            0.101435
NEOPLASIAS (140-239)                                                            0.086417
ENFERMEDADES DEL APARATO GENITOURINARIO (580-629)                               0.085158
SÍNTOMAS, SIGNOS Y ESTADOS MAL DEFINIDOS (780-799)                              0.075258
ENFERMEDADES DEL SISTEMA NERVIOSO Y DE LOS ÓRGANOS DE LOS SENTIDOS (320-389)    0.056464
Name: GENERAL_DIAG, dtype: float64

In [0]:
df_final = med_df[['MOTIVO_CONSULTA','GENERAL_DIAG']]

### Parte de preprocesamiento de texto.

In [100]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import re
import string
from gensim import corpora, models
import gensim
import unicodedata

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def find_age(string):
  pattern = r'(\d+)\s?años'
  if len(re.findall(pattern, string)) >0:
    age = int(re.findall(pattern, string)[0])
    if age <= 15:
      age_group = 'niño'
    elif 15 < age <= 30:
      age_group = 'joven'
    elif 30 < age <= 45:
      age_group = 'adulto'
    elif 45 < age <= 60:
      age_group = 'maduro'
    elif 60 < age <= 75:
      age_group = 'mayor'
    else:
      age_group = 'anciano'
    string = re.sub(pattern, age_group, string)
    string = re.sub('años', '', string)

  else:
    string = string
  return string

In [0]:
def strip_accents(text):
  
    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text)


In [0]:
def strip_accents_spain(string, accents=('COMBINING ACUTE ACCENT', 'COMBINING GRAVE ACCENT')):
    accents = set(map(unicodedata.lookup, accents))
    chars = [c for c in unicodedata.normalize('NFD', string) if c not in accents]
    return unicodedata.normalize('NFC', ''.join(chars))

In [0]:
def text_prepro(cadena):
  
  cadena = cadena.lower()
  cadena = find_age(cadena).replace('edad', '')
  cadena = strip_accents(cadena)
  result = re.sub(r'\d+', '', cadena)
  result = result.translate(str.maketrans('', '', string.punctuation))
  result = result.strip()
  stop_words = set(stopwords.words('spanish'))
  word_tokens = word_tokenize(result) 
  filtered_sentence = [w for w in word_tokens if not w in stop_words and len(w) > 2]
  lemmatizer=WordNetLemmatizer()
  lematized_words = [lemmatizer.lemmatize(word) for word in filtered_sentence]
  stemmer= PorterStemmer()
  stemmed_words = [stemmer.stem(word) for word in lematized_words]
  stemmed_string = ' '.join(stemmed_words)
  return stemmed_string

In [389]:
df_final['MOTIVO_TRANS'] = df_final['MOTIVO_CONSULTA'].apply(lambda x: text_prepro(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [390]:
df_final[['MOTIVO_TRANS', 'MOTIVO_CONSULTA']].head(5)

,MOTIVO_TRANS,MOTIVO_CONSULTA
0,pacient mayor derivada servicio digestivo hud ...,Paciente de 74 años de edad que es derivada de...
1,mujer mayor ingresa procedent onkologikoa npt ...,"Mujer de 70años, ingresa procedente de Onkolog..."
2,pacient acud presentar traumatismo pelvi tra c...,paciente que acude por presentar traumatis...
3,ingresa forma programada tratamiento quirurgic...,Ingresa de forma programada para tratamiento q...
4,ingresa forma programada tratamiento quirurgic...,Ingresa de forma programada para tratamiento q...


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import f1_score

In [0]:
X = df_final['MOTIVO_TRANS']
y = df_final['GENERAL_DIAG']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state = 42)

Analizamos las palabras por su frecuencia.

In [0]:
vec = CountVectorizer().fit(df_final['MOTIVO_TRANS'])

In [401]:
2131/len(df_final['MOTIVO_TRANS'])

0.17879016696031547

In [0]:
bag_of_words = vec.transform(df_final['MOTIVO_TRANS'])

In [0]:
sum_words = bag_of_words.sum(axis=0)

In [0]:
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]

In [0]:
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

In [440]:
words_freq[:16]

[('ingresa', 5880),
 ('tratamiento', 4964),
 ('pacient', 3870),
 ('programada', 3501),
 ('tra', 2836),
 ('intervencion', 2755),
 ('forma', 2669),
 ('estudio', 2651),
 ('consentimiento', 2550),
 ('informado', 2539),
 ('preoperatorio', 2538),
 ('riesgo', 2537),
 ('firma', 2531),
 ('explicado', 2530),
 ('asi', 2526),
 ('serl', 2522)]

In [441]:
2522/len(df_final['MOTIVO_TRANS'])

0.2115949324607769

### Naive Bayes model pipeline

In [447]:
nb = Pipeline([('vect', CountVectorizer(max_df = 0.3, min_df = 0.00000001)),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.3,
                                 max_features=None, min_df=1e-08,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
y_pred = nb.predict(X_test)

### Los resultados del modelo.

In [444]:
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.6778523489932886


In [445]:
print(classification_report(y_test, y_pred))

                                                                              precision    recall  f1-score   support

                                ENFERMEDADES DEL APARATO DIGESTIVO (520-579)       0.69      0.84      0.76        69
                           ENFERMEDADES DEL APARATO GENITOURINARIO (580-629)       0.67      0.29      0.41        48
                             ENFERMEDADES DEL APARATO RESPIRATORIO (460-519)       0.56      0.91      0.70       113
                             ENFERMEDADES DEL SISTEMA CIRCULATORIO (390-459)       0.51      0.55      0.53        80
ENFERMEDADES DEL SISTEMA NERVIOSO Y DE LOS ÓRGANOS DE LOS SENTIDOS (320-389)       0.95      0.61      0.74        33
   ENFERMEDADES DEL SISTEMA OSTEO-MIOARTICULAR Y TEJIDO CONJUNTIVO (710-739)       0.89      0.89      0.89       109
                                        LESIONES Y ENVENENAMIENTOS (800-999)       0.82      0.71      0.76        52
                                                       

In [449]:
f1_score(y_test, y_pred, average='macro')

0.6219838892044569

### Aplicacion y serializacion del modelo.

In [434]:
joblib.dump(nb, '/content/drive/My Drive/Datos_medicos/nb_model.sav')

['/content/drive/My Drive/Datos_medicos/nb_model.sav']

In [0]:
def text_to_class(text):
  
  loaded_model = joblib.load('/content/drive/My Drive/Datos_medicos/nb_model.sav')

  text = text_prepro(text)
  prueba = pd.Series(text)
  output = loaded_model.predict(prueba)

  return output[0].split('(')[0].strip()

In [377]:
text_to_class('ingresa forma programada tratamiento quirúrgico enf dupuytren º dedo mano izquierda prevista realización fasciotomía palmar tra estudio preoperatorio serl explicado consist intervención riesgo así alternativa tratamiento firma consentimiento informado')

'ENFERMEDADES DEL SISTEMA OSTEO-MIOARTICULAR Y TEJIDO CONJUNTIVO'